In [2]:
import urllib.request, urllib.parse, urllib.error
import numpy as np
import pandas as pd
pd.options.display.max_columns=50

#import PyQt5
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
import sys

from numpy.random import seed
seed(888)
import os

from sklearn.metrics import r2_score
from sklearn.metrics import  mean_absolute_error 
from catboost import CatBoostRegressor,Pool
import lightgbm as lgb

from scipy import stats
from scipy.cluster.vq import vq, kmeans, whiten

# !pip install wget
import wget

# !pip install missingno
import missingno as msno

# !pip install umap-learn
# !pip install hdbscan


# !pip install pandas_profiling


# !/usr/bin/python3 -m pip install --upgrade pip


import holidays
%matplotlib inline

In [3]:
df = pd.read_csv('test.csv')

In [4]:
df_train_2 = df.copy(deep=True)
# df_train_2 = df_train_2.sample(10000000)

In [5]:
df_train_2 = df_train_2.loc[~(df_train_2['created']<='2021-01-01')]
df_train_2

,gamecategory,subgamecategory,bundle,created,shift,oblast,city,os,osv
0,Applications,Shopping,com.allgoritm.youla,2021-09-19 17:31:33,MSK+2,Свердловская область,Екатеринбург,android,10.0
1,NaN,NaN,com.ChocochocoStd.RapBattle,2021-08-03 17:13:17,MSK+2,Ямало-Ненецкий АО,Новый Уренгой,android,10.0
2,NaN,NaN,1387897651,2021-09-17 15:54:00,MSK,Москва,Москва,ios,14.4.0
3,NaN,NaN,com.cooking.family.diary.fever.food.city.craze...,2021-07-05 23:34:59,MSK,Краснодарский край,Краснодар,android,9.0.0
4,Applications,Health & Fitness,com.pedometer.stepcounter.tracker,2021-07-08 15:15:21,MSK,Татарстан,Набережные Челны,android,10.0
...,...,...,...,...,...,...,...,...,...
11213624,NaN,NaN,com.phonemaster.jewelhunter,2021-08-04 05:55:26,MSK,Санкт-Петербург,Санкт-Петербург,android,8.1.0
11213625,NaN,NaN,com.linkdesks.jewellegend,2021-09-17 21:45:02,MSK,Ставрополье,Пятигорск,android,10
11213626,Applications,Health & Fitness,com.pedometer.stepcounter.tracker,2021-07-08 10:24:31,MSK+6,Забайкальский Край,Чита,android,11.0
11213627,NaN,NaN,com.onebutton.mrsuper,2021-07-05 16:02:47,MSK+2,Башкортостан,Белорецк,android,6.0.1


In [6]:
#
#ALERT
df_train_2['created'] = df_train_2['created'].fillna(method='ffill')
#ALERT
#

df_train_2['created'] = pd.to_datetime(df_train_2['created'], format = '%Y-%m-%d %H:%M:%S')
df_train_2['year'] = df_train_2['created'].dt.year
df_train_2['month'] = df_train_2['created'].dt.month
df_train_2['day'] = df_train_2['created'].dt.day
df_train_2['hour'] = df_train_2['created'].dt.hour



df_train_2

,gamecategory,subgamecategory,bundle,created,shift,oblast,city,os,osv,year,month,day,hour
0,Applications,Shopping,com.allgoritm.youla,2021-09-19 17:31:33,MSK+2,Свердловская область,Екатеринбург,android,10.0,2021,9,19,17
1,NaN,NaN,com.ChocochocoStd.RapBattle,2021-08-03 17:13:17,MSK+2,Ямало-Ненецкий АО,Новый Уренгой,android,10.0,2021,8,3,17
2,NaN,NaN,1387897651,2021-09-17 15:54:00,MSK,Москва,Москва,ios,14.4.0,2021,9,17,15
3,NaN,NaN,com.cooking.family.diary.fever.food.city.craze...,2021-07-05 23:34:59,MSK,Краснодарский край,Краснодар,android,9.0.0,2021,7,5,23
4,Applications,Health & Fitness,com.pedometer.stepcounter.tracker,2021-07-08 15:15:21,MSK,Татарстан,Набережные Челны,android,10.0,2021,7,8,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11213624,NaN,NaN,com.phonemaster.jewelhunter,2021-08-04 05:55:26,MSK,Санкт-Петербург,Санкт-Петербург,android,8.1.0,2021,8,4,5
11213625,NaN,NaN,com.linkdesks.jewellegend,2021-09-17 21:45:02,MSK,Ставрополье,Пятигорск,android,10,2021,9,17,21
11213626,Applications,Health & Fitness,com.pedometer.stepcounter.tracker,2021-07-08 10:24:31,MSK+6,Забайкальский Край,Чита,android,11.0,2021,7,8,10
11213627,NaN,NaN,com.onebutton.mrsuper,2021-07-05 16:02:47,MSK+2,Башкортостан,Белорецк,android,6.0.1,2021,7,5,16


In [7]:

df_train_2[['created_sm','to_drop']] = df_train_2['created'].astype(str).str.split(' ',expand=True)
df_train_2 = df_train_2.drop(columns = 'to_drop')
df_train_2['created_sm'] = pd.to_datetime(df_train_2['created_sm'])
df_train_2

,gamecategory,subgamecategory,bundle,created,shift,oblast,city,os,osv,year,month,day,hour,created_sm
0,Applications,Shopping,com.allgoritm.youla,2021-09-19 17:31:33,MSK+2,Свердловская область,Екатеринбург,android,10.0,2021,9,19,17,2021-09-19
1,NaN,NaN,com.ChocochocoStd.RapBattle,2021-08-03 17:13:17,MSK+2,Ямало-Ненецкий АО,Новый Уренгой,android,10.0,2021,8,3,17,2021-08-03
2,NaN,NaN,1387897651,2021-09-17 15:54:00,MSK,Москва,Москва,ios,14.4.0,2021,9,17,15,2021-09-17
3,NaN,NaN,com.cooking.family.diary.fever.food.city.craze...,2021-07-05 23:34:59,MSK,Краснодарский край,Краснодар,android,9.0.0,2021,7,5,23,2021-07-05
4,Applications,Health & Fitness,com.pedometer.stepcounter.tracker,2021-07-08 15:15:21,MSK,Татарстан,Набережные Челны,android,10.0,2021,7,8,15,2021-07-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11213624,NaN,NaN,com.phonemaster.jewelhunter,2021-08-04 05:55:26,MSK,Санкт-Петербург,Санкт-Петербург,android,8.1.0,2021,8,4,5,2021-08-04
11213625,NaN,NaN,com.linkdesks.jewellegend,2021-09-17 21:45:02,MSK,Ставрополье,Пятигорск,android,10,2021,9,17,21,2021-09-17
11213626,Applications,Health & Fitness,com.pedometer.stepcounter.tracker,2021-07-08 10:24:31,MSK+6,Забайкальский Край,Чита,android,11.0,2021,7,8,10,2021-07-08
11213627,NaN,NaN,com.onebutton.mrsuper,2021-07-05 16:02:47,MSK+2,Башкортостан,Белорецк,android,6.0.1,2021,7,5,16,2021-07-05


In [13]:
NON_WORK_DAY_DICT = {
    2021: {
        1: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 16, 17, 23, 24, 30, 31],
        2: [6, 7, 13, 14, 21, 22, 23, 27, 28],
        3: [6, 7, 8, 13, 14, 20, 21, 27, 28],
        4: [3, 4, 10, 11, 17, 18, 24, 25],
        5: [1, 2, 3, 8, 9, 10, 15, 16, 22, 23, 29, 30],
        6: [5, 6, 12, 13, 14, 19, 20, 26, 27],
        7: [3, 4, 10, 11, 17, 18, 24, 25, 31],
        8: [1, 7, 8, 14, 15, 21, 22, 28, 29],
        9: [4, 5, 11, 12, 18, 19, 25, 26],
        10: [2, 3, 9, 10, 16, 17, 23, 24, 30, 31],
        11: [4, 5, 6, 7, 13, 14, 20, 21, 27, 28],
        12: [4, 5, 11, 12, 18, 19, 25, 26, 31]
    },
    2022: {
        1: [1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 16, 22, 23, 29, 30],
        2: [5, 6, 12, 13, 19, 20, 23, 26, 27],
        3: [6, 7, 8, 12, 13, 19, 20, 26, 27],
        4: [2, 3, 9, 10, 16, 17, 23, 24, 30],
        5: [1, 2, 3, 7, 8, 9, 10, 14, 15, 21, 22, 28, 29],
        6: [4, 5, 11, 12, 13, 18, 19, 25, 26],
        7: [2, 3, 9, 10, 16, 17, 23, 24, 30, 31],
        8: [6, 7, 13, 14, 20, 21, 27, 28],
        9: [3, 4, 10, 11, 17, 18, 24, 25],
        10: [1, 2, 8, 9, 15, 16, 22, 23, 29, 30],
        11: [4, 5, 6, 12, 13, 19, 20, 26, 27],
        12: [3, 4, 10, 11, 17, 18, 24, 25, 31]
    }
}


In [35]:
df_train_2['is_working'] = 0

for i in df_train_2.index:
    if not df_train_2['day'].loc[i] in NON_WORK_DAY_DICT[df_train_2['year'].loc[i]][df_train_2['month'].loc[i]]:
        df_train_2['is_working'].loc[i] = 1

df_train_2

In [18]:
df_train_2

,gamecategory,subgamecategory,bundle,created,shift,oblast,city,os,osv,year,month,day,hour,created_sm,is_working
0,0,40,15843,1524199,2,69,590,1,3,0,2,6,9,18,0
1,27,52,11409,703119,2,89,1471,1,3,0,1,9,9,8,0
2,27,52,2537,1350896,0,48,1280,3,115,0,2,4,7,16,0
3,27,52,21070,82023,0,36,984,1,212,0,0,11,16,0,0
4,0,23,38962,288582,0,74,1313,1,3,0,0,14,7,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11213624,27,52,39147,747074,0,65,1819,1,202,0,1,10,19,9,0
11213625,27,52,33299,1371855,0,72,1729,1,1,0,2,4,14,16,0
11213626,0,23,38962,271596,6,21,2325,1,23,0,0,14,2,3,0
11213627,27,52,38179,55985,2,9,180,1,181,0,0,11,8,0,0


In [17]:
import ephem,datetime

sun = ephem.Sun()
moon = ephem.Moon()
raleigh = ephem.Observer()
raleigh.lon, raleigh.lat = "36.8251405", "55.5815245" #msk


# raleigh.lon, raleigh.lat = "134.973", "48.4649" #хабар

raleigh.horizon = '-0:34'  # USNO standard atmospheric diffraction
raleigh.pressure = 0       # atmospheric refraction parameters

def riseset_sun(date, f):
    raleigh.date = date
    sun.compute(raleigh)
    sr = ephem.localtime(f(sun))    
    return sr

def riseset_moon(date, f):
    raleigh.date = date
    moon.compute(raleigh)
    sr = ephem.localtime(f(moon))    
    return sr



def sunframe(df):
    #Солнце
    df['sunrise'] = df['created_sm'].map(lambda d: riseset_sun(d, raleigh.next_rising)) #+ np.timedelta64(7, 'h')
    df['sunset'] = df['created_sm'].map(lambda d: riseset_sun(d, raleigh.next_setting)) #+ np.timedelta64(7, 'h')
    
    df['sunrise_h']=df['sunrise'].dt.hour
    df['sunrise_m']=df['sunrise'].dt.minute/60
    
    df['sunset_h']=df['sunset'].dt.hour      
    df['sunset_m']=df['sunset'].dt.minute/60

    
    df['Dsun']=(df['sunset_h'] + df['sunset_m'])-(df['sunrise_h'] + df['sunrise_m'])
    df['sunrise']=df['sunset_h'] + df['sunset_m']
    df['sunset'] =df['sunrise_h'] + df['sunrise_m']
    

    #луна 5-10% к точности
    df['sunrise_moon'] = df['created_sm'].map(lambda d: riseset_moon(d, raleigh.next_rising)) #+ np.timedelta64(7, 'h')
    df['sunset_moon'] = df['created_sm'].map(lambda d: riseset_moon(d, raleigh.next_setting)) #+ np.timedelta64(7, 'h')

    df['sunrise_moon_h']=df['sunrise_moon'].dt.hour
    df['sunrise_moon_m']=df['sunrise_moon'].dt.minute/60
    
    df['sunset_moon_h']=df['sunset_moon'].dt.hour      
    df['sunset_moon_m']=df['sunset_moon'].dt.minute/60
    
    df['sunrise_moon']=df['sunrise_moon_h']+df['sunrise_moon_m']
    df['sunset_moon']=df['sunset_moon_h']+df['sunset_moon_m']
    
    df['Dmoon']=df['sunset_moon']-df['sunrise_moon']
    
    
    
    return df

In [ ]:
# df_train_2 = sunframe(df_train_2)
# df_train_2

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for i in range(0,len(df_train_2.columns)):
    
    df_train_2[(df_train_2.columns[i])] = le.fit_transform(df_train_2[[(df_train_2.columns[i])]].astype(str))

/home/zodiac180/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


## Catboost train

In [23]:
X = df_train_2.values

In [10]:
from catboost import CatBoostClassifier, Pool

In [25]:
from_file = CatBoostClassifier(task_type = 'GPU')

model = from_file.load_model('nds_catboost.model')

y = model.predict(X)

In [27]:
y.shape

(11213560, 1)

In [34]:
import pandas as pd
pd.DataFrame(y).to_csv('results.csv',index=False)
